# Chapter 8. Dimensionality Reduction

In various Machine Learning scenarios, the inclusion of numerous features per training instance can lead to slow and challenging training processes due to the curse of dimensionality. However, this predicament can often be mitigated in real-world applications by substantially reducing feature counts, thus rendering an initially intractable task more manageable. For instance, in the case of MNIST images, pixels situated along the image periphery can typically be disregarded without significant loss of information. 

Similarly, the correlation between neighboring pixels permits their consolidation into single pixels, contributing to dimensionality reduction. It is important to note that although dimensionality reduction accelerates training, it may marginally affect system performance due to information loss and increased pipeline complexity. Dimensionality reduction's benefits extend beyond hastening training, as it enables effective data visualization, condensing multidimensional datasets into two or three dimensions for graphical representation, aiding pattern recognition and insights.

Additionally, this form of visualization, termed DataViz, serves as a critical tool for conveying findings to non-data scientist stakeholders, especially decision-makers who will utilize the outcomes. This chapter explores the intricacies of the curse of dimensionality and delves into the dynamics of high-dimensional space. Subsequently, it delves into the two core dimensionality reduction methodologies—projection and Manifold Learning. Three prominent techniques for dimensionality reduction—PCA, Kernel PCA, and LLE—are thoroughly examined and discussed, elucidating their applications and impact.

### The Curse of Dimensionality

Our cognitive understanding falters when we attempt to envision high-dimensional spaces, where intuition struggles to grasp the intricacies. Even conceiving a basic 4D hypercube becomes complex, let alone envisaging more dimensions. In such spaces, various phenomena diverge from our accustomed perception. For instance, the behavior of distances between points transforms substantially. In a low-dimensional context like a unit square or cube, randomly chosen points exhibit relatively short average distances between them. However, in a high-dimensional realm, like a 1,000,000-dimensional hypercube, the average distance between two randomly selected points astonishingly stretches out, reflecting the abundant expanse in high dimensions. Consequently, high-dimensional datasets often fall victim to sparsity, rendering training instances widely scattered, thereby undermining the reliability of predictions due to substantial extrapolations.

One potential solution to the challenges posed by high-dimensional spaces is to augment the training set's size, aiming to achieve an adequate density of instances. Nonetheless, practical implementation highlights a daunting reality: the number of training instances required grows exponentially with dimensionality. In the case of just 100 features, considerably fewer than in complex tasks like the MNIST problem, achieving an average distance of 0.1 between instances necessitates more instances than the observable universe's atoms, assuming even distribution across dimensions. This curse of dimensionality underscores the formidable complexity introduced by the amplification of dimensions and the subsequent exponential increase in the demand for training data.

### Main Approaches for Dimensionality Reduction


Two primary approaches to dimensionality reduction, projection, and Manifold Learning, are essential concepts for addressing the curse of dimensionality. Projection involves identifying a lower-dimensional subspace within which training instances predominantly reside, driven by the observation that real-world instances are not uniformly distributed across all dimensions. This is illustrated by projecting a 3D dataset onto a 2D subspace, where instances lie close to a plane, thus reducing dimensionality. 

However, projecting might not be ideal when the subspace twists, as shown by the Swiss roll dataset, where simple projection would distort the data's inherent structure. In contrast, Manifold Learning deals with 2D or d-dimensional manifolds that can be flexibly contorted within a higher-dimensional space. Algorithms in this category aim to model the manifold underlying training instances. The manifold assumption posits that high-dimensional datasets tend to lie near lower-dimensional manifolds, often observed empirically, driven by inherent constraints on data generation. This assumption implies that expressing tasks in a lower-dimensional manifold space could simplify problem-solving. 

This assumption's applicability can vary; while some tasks may indeed be simpler in the manifold space, others could experience increased complexity, contingent on the specific dataset. Dimensionality reduction, though generally beneficial for speeding up model training, may not universally lead to better or simpler solutions, the outcome being contingent upon the dataset's nature.

### PCA

Principal Component Analysis (PCA) stands as a widely adopted dimensionality reduction technique. It operates by initially identifying the hyperplane that exhibits the nearest proximity to the dataset. Following this, PCA proceeds to project the data onto this identified hyperplane. The core principle underlying PCA involves extracting orthogonal axes known as principal components from the original feature space, prioritizing those axes that capture maximum variance in the data. 

These components correspond to directions that effectively represent the dataset's variability. PCA's primary objective is to transform the data into a new coordinate system, where the first principal component aligns with the direction of greatest variance, the second principal component becomes orthogonal to the first and captures the next highest variance, and so on. This transformation results in reduced dimensions, allowing for a compact representation of the data while preserving the most significant variance.

In summary, PCA seeks to simplify the complexity of high-dimensional data by projecting it onto a lower-dimensional subspace defined by principal components. These components are chosen to encapsulate the most substantial variability within the dataset, enabling a streamlined representation that retains essential information while shedding the less informative dimensions.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "dim_reduction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Building a simple 3D dataset:

In [2]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

Principal Components

In [3]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [4]:
m, n = X.shape

S = np.zeros(X_centered.shape)
S[:n, :n] = np.diag(s)

In [5]:
np.allclose(X_centered, U.dot(S).dot(Vt))


True

Projecting Down to d Dimensions

In [6]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

In [7]:
X2D_using_svd = X2D


Using Scikit-Learn

In [10]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [11]:
X2D[:5]


array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049]])

In [12]:
X2D_using_svd[:5]


array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])


Notice that running PCA multiple times on slightly different datasets may result in different results. In general the only difference is that some axes may be flipped. In this example, PCA using Scikit-Learn gives the same projection as the one given by the SVD approach, except both axes are flipped:

In [13]:
np.allclose(X2D, -X2D_using_svd)


True